# Plot data distributions
Due to some issues of `cartopy` to work in Julia, the plots are prepared in this notebook in Python.

In [1]:
import os
import netCDF4
import toml
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature 
import cartopy.mpl.ticker as cartopyticker
coast = cfeature.GSHHSFeature(scale="high")
lon_formatter = cartopyticker.LongitudeFormatter()
lat_formatter = cartopyticker.LatitudeFormatter()
plt.style.use("calanus.mplstyle")

<frozen importlib._bootstrap>:219: UserWarning: PROJ 8+ is required. Current version: 7.2.1


## Settings
### Read configuration file

In [2]:
config = toml.load("config.ini")
config

{'title': 'Calanus interpolation parameters',
 'domain': [-20.5, 11.75, 41.25, 67.0],
 'dlon': 0.25,
 'dlat': 0.25,
 'directories': {'datadir': '../data/',
  'figdir': '../figures/',
  'resfigdir': '../figures/L025/',
  'resdir': '../results/L025/'},
 'files': {'datafile': 'MBA_CPRdata_Emodnet_21Jan22.csv',
  'datafileURL': 'https://dox.ulg.ac.be/index.php/s/hjWKf1F3C1Pzz1r/download',
  'bathyfile': 'gebco_30sec_16.nc'}}

### Domain and projections

In [3]:
dom = config["domain"]
mainproj = ccrs.Mercator(central_longitude=0.5*(dom[0] + dom[1]),
    min_latitude=dom[2], max_latitude=dom[3])
datacrs = ccrs.PlateCarree();
mainproj

<Projected CRS: +proj=merc +ellps=WGS84 +lon_0=-4.375 +x_0=0.0 +y_ ...>
Name: unknown
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- undefined
Coordinate Operation:
- name: unknown
- method: Mercator (variant A)
Datum: Unknown based on WGS84 ellipsoid
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

### Directories and files

In [4]:
datadir = config["directories"]["datadir"]
figdir = config["directories"]["figdir"]
resdir = config["directories"]["resdir"]
datafile = os.path.join(datadir, "MBA_CPRdata_Emodnet_21Jan22.csv")
woafile = "../data/WOD/temperature_surface_WOD2.nc"
bathfile = os.path.join(datadir, "gebco_30sec_16.nc")
outputfile1 = os.path.join(resdir, "calanus_finmarchicus_year_month.nc")
outputfile2 = os.path.join(resdir, "calanus_helgolandicus_year_month.nc")

## Load observations

In [5]:
def read_data_csv(datafile):
    lon = []
    lat = []
    year = []
    month = []
    calanusfinmarchicus = []
    calanushelgolandicus = []
    with open(datafile, "r") as df:
        df.readline()
        for lines in df:
            lsplit = lines.rstrip().split(',')
            lat.append(float(lsplit[1]))
            lon.append(float(lsplit[2]))
            year.append(int(lsplit[3]))
            month.append(int(lsplit[4]))
            calanusfinmarchicus.append(float(lsplit[5]))
            calanushelgolandicus.append(float(lsplit[6]))
            
    return np.array(lon), np.array(lat), np.array(year), np.array(month), np.array(calanusfinmarchicus), np.array(calanushelgolandicus)

In [6]:
lonobs, latobs, yearobs, monthobs, calanusfinmarchicus, calanushelgolandicus = read_data_csv(datafile)

## Plots
### All observations

In [7]:
fig = plt.figure(figsize=(12, 12))
ax = plt.subplot(111, projection=mainproj)
ax.plot(lonobs, latobs, "ko", markersize=1, transform=datacrs)
ax.add_feature(coast, facecolor="#363636", edgecolor="k", zorder=5)

gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                      linewidth=.5, color="gray", alpha=1, linestyle="--")
gl.top_labels = False
gl.right_labels = False
gl.xformatter = lon_formatter
gl.yformatter = lat_formatter
ax.set_title("Locations of the observations")
plt.savefig(os.path.join(figdir, "location_obs"))
plt.close()

### Hexbin plots

In [8]:
fig = plt.figure(figsize=(12, 12))
ax = plt.subplot(111, projection=mainproj)
hb = ax.hexbin(lonobs, latobs, transform=datacrs, cmap=plt.cm.hot_r, gridsize=70, vmax=50)
cb = plt.colorbar(hb, extend="max")
ax.add_feature(coast, facecolor="#363636", edgecolor="k", zorder=5)
ax.set_extent(dom)
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                      linewidth=.5, color="gray", alpha=1, linestyle="--")
gl.top_labels = False
gl.right_labels = False
gl.xformatter = lon_formatter
gl.yformatter = lat_formatter
ax.set_title("Density of observations")
plt.savefig(os.path.join(figdir, "location_hexbin"))
plt.close()

### Scatter plots by species

In [9]:
countmin = 5
fig = plt.figure(figsize=(12, 12))
ax = plt.subplot(111, projection=mainproj)
ax.set_extent(dom)
ax.scatter(lonobs, latobs, s=.1, lw=.1, color="k", transform=datacrs)
indexplot = np.where(calanusfinmarchicus >= countmin)[0]
scat = ax.scatter(lonobs[indexplot], latobs[indexplot], s=1, c=calanusfinmarchicus[indexplot], 
                  cmap=plt.cm.inferno_r, vmin=countmin, vmax=50., transform=datacrs)

cb = plt.colorbar(scat, extend="both")

ax.add_feature(coast, facecolor="#363636", edgecolor="k", zorder=5)

gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                      linewidth=.5, color="gray", alpha=1, linestyle="--")
gl.top_labels = False
gl.right_labels = False
gl.xformatter = lon_formatter
gl.yformatter = lat_formatter
ax.set_title("Calanus finmarchicus observations")

plt.savefig(os.path.join(figdir, "count_calanus_finmarchicus2"))
plt.close()

In [10]:
countmin = 5
fig = plt.figure(figsize=(12, 12))
ax = plt.subplot(111, projection=mainproj)
ax.scatter(lonobs, latobs, s=.1, lw=.1, color="k", transform=datacrs)
ax.set_extent(dom)
indexplot = np.where(calanushelgolandicus >= countmin)[0]
scat = ax.scatter(lonobs[indexplot], latobs[indexplot], s=1, c=calanushelgolandicus[indexplot], 
                  cmap=plt.cm.inferno_r, vmin=countmin, vmax=50., transform=datacrs)

cb = plt.colorbar(scat, extend="both")

ax.add_feature(coast, facecolor="#363636", edgecolor="k", zorder=5)

gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                      linewidth=.5, color="gray", alpha=1, linestyle="--")
gl.top_labels = False
gl.right_labels = False
gl.xformatter = lon_formatter
gl.yformatter = lat_formatter
ax.set_title("Calanus helgolandicus observations")

plt.savefig(os.path.join(figdir, "count_calanus_helgolandicus2.jpg"))
plt.close()

### Hexbin plots by species

In [11]:
fig = plt.figure(figsize=(12, 12))
ax = plt.subplot(111, projection=mainproj)
ax.set_extent(dom)
hb = ax.hexbin(lonobs, latobs, C=calanusfinmarchicus, transform=datacrs, 
               cmap=plt.cm.hot_r, gridsize=75, vmax=75.)
cb = plt.colorbar(hb, extend="max")
ax.add_feature(coast, facecolor="#363636", edgecolor="k", zorder=5)

gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                      linewidth=.5, color="gray", alpha=1, linestyle="--")
gl.top_labels = False
gl.right_labels = False
gl.xformatter = lon_formatter
gl.yformatter = lat_formatter
ax.set_title("Calanus finmarchicus hexbin map")
plt.savefig(os.path.join(figdir, "count_calanus_finmarchicus_hexbin"))
plt.close()

In [12]:
fig = plt.figure(figsize=(12, 12))
ax = plt.subplot(111, projection=mainproj)
ax.set_extent(dom)
hb = ax.hexbin(lonobs, latobs, C=calanushelgolandicus, transform=datacrs, 
               cmap=plt.cm.hot_r, gridsize=75, vmax=75.)
cb = plt.colorbar(hb, extend="max")
ax.add_feature(coast, facecolor="#363636", edgecolor="k", zorder=5)

gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                      linewidth=.5, color="gray", alpha=1, linestyle="--")
gl.top_labels = False
gl.right_labels = False
gl.xformatter = lon_formatter
gl.yformatter = lat_formatter
ax.set_title("Calanus helgolandicus hexbin map")
plt.savefig(os.path.join(figdir, "count_calanus_helgolandicus_hexbin.jpg"))
plt.close()